In [ ]:
import numpy as np # manipulação de matrizes
import pandas as pd # Manipulação de data/sets
import matplotlib.pyplot as plt # Para plotagem
import scipy.stats as stats # Normalizações, cálculos de densidade de probabilidade, etc

In [ ]:
## Lendo o dataset e processando os dados
dataBTC = pd.read_csv("data/bitcoin_price.csv")

def getRelativeChange(ticker):
    return (ticker['Close'] - ticker['Open']) / ticker['Open']

variationBTC = dataBTC.apply(getRelativeChange, axis=1)

### Histograma

In [ ]:
plt.hist(variationBTC, 100, normed=1)

plt.xlabel('% Change')
plt.ylabel('% Probability')
plt.title(r'$\mathrm{Histogram\ of\ BTC}$')
plt.grid(True)
plt.show()

### Média, Variância e Desvio Padrão Amostral

In [ ]:
totalVariation = variationBTC.sum()
mean = totalVariation/len(variationBTC)
print("{0:15} {1}".format('Média', mean))

# unbiased variance
variance = variationBTC.var()
print("{0:15} {1}".format('Variância', variance))

std = (variance)**(1/2)
print("{0:15} {1}".format('Desvio Padrão', std))